In [10]:
import numpy as np
from pandas import read_csv
from pandas import to_datetime
from sklearn.preprocessing import MinMaxScaler
import math

# load the train data
dataframe = read_csv('dataset/train.csv',usecols=[i for i in range(0,211)], engine='python', header=0)
trainData = dataframe.values
trainDataOriginal = np.nan_to_num(trainData)
trainData = trainDataOriginal
# load the test data
dataframe = read_csv('dataset/test_2.csv',usecols=[i for i in range(0,146)], engine='python', header=0)
testData = dataframe.values
testData = np.nan_to_num(testData)
testX = testData

In [11]:
#parameters
batch_size = 64
input_dim  = 146
output_dim = 64

In [12]:
#Separate the train data to input and output
trainX = trainData[0:len(trainData),0:146]
trainY = trainData[0:len(trainData),147:211]
# normalize the dataset
scalerInput = MinMaxScaler(feature_range=(0, 1))
scalerOutput = MinMaxScaler(feature_range=(0, 1))

trainX = scalerInput.fit_transform(trainX)
trainY = scalerOutput.fit_transform(trainY)
testX = scalerInput.fit_transform(testX)

#Put it into batches
trainX = np.reshape(trainX,(len(trainX)/batch_size,batch_size,trainX.shape[1]))
trainY = np.reshape(trainY,(len(trainY)/batch_size,batch_size,trainY.shape[1]))
testX  = np.reshape(testX,(len(testX)/batch_size,batch_size,testX.shape[1]))

print trainX.shape
print trainY.shape
print testX.shape

(625, 64, 146)
(625, 64, 64)
(1875, 64, 146)


In [27]:
from keras.activations import linear
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import adam
from sklearn.metrics import mean_squared_error
from keras.models import Model

class Model:
    
    def __init__(self, trainX, trainY):
        self.optimizer = adam(0.01, 0.01, 0.001, 1e-08, 0.0)
        self.model = Sequential()
        self.model.add(LSTM(output_dim, input_shape=(batch_size, input_dim), return_sequences=True))
        self.model.add(Dense(output_dim, activation='linear'))
        #Configures the model for training.
        self.model.compile(loss="mean_squared_error",optimizer=self.optimizer)
        #Trains the model for a fixed number of epochs (iterations on a dataset).
        self.model.fit(trainX,trainY,batch_size=batch_size,epochs=1)
        #Make predictions
    
    def predict(self,testX):
        return self.model.predict(testX)
    

In [28]:
#train the model
nn = Model(trainX, trainY)
testY = nn.predict(testX)

Epoch 1/1
625/625 [==============================] - 2s - loss: 0.0907     


In [29]:
testY = testY.reshape(len(testData),output_dim)
testY = scalerOutput.inverse_transform(testY)
print testY


[[ -1.04213767e-02  -3.48009123e-03  -1.35742286e-02 ...,  -1.68686584e-01
    1.34140275e+06   1.93409975e+06]
 [ -1.02256797e-02   1.74950750e-03  -7.79836020e-03 ...,  -7.45808929e-02
    1.44399075e+06   2.17367225e+06]
 [ -1.02537451e-02   3.20519414e-03  -6.34601293e-03 ...,  -2.39081196e-02
    1.48224100e+06   2.24608675e+06]
 ..., 
 [ -9.90756508e-03   2.96240509e-03  -4.44969628e-03 ...,   8.56180862e-03
    1.56963175e+06   2.29875975e+06]
 [ -8.35386757e-03   2.28425325e-03  -5.56152174e-03 ...,   2.34614667e-02
    1.54332400e+06   2.33083375e+06]
 [ -9.74762160e-03   2.52898969e-03  -3.77519336e-03 ...,   1.53078828e-02
    1.53800838e+06   2.31886775e+06]]
